# Comprehensive Time Series Analysis Example

This notebook demonstrates the full capabilities of the specialized-viz time series module, including:
1. Basic Time Series Analysis
2. Seasonal Decomposition
3. Pattern Detection
4. Advanced Analytics
5. Forecasting
6. Visualization

Let's start by importing the necessary libraries and setting up our environment.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
from specialized_viz.timeseries import (
    TimeseriesAnalysis,
    TimeseriesConfig,
    TimeseriesVisualizer,
    TimeseriesForecasting
)

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Initialize analyzer with configuration
config = TimeseriesConfig(
    decomposition_method='additive',
    seasonal_periods=[5, 21, 63, 252],  # Daily, Weekly, Monthly, Yearly
    trend_window=20,
    forecast_horizon=30,
    cycle_max_period=252  # One trading year
)

## 1. Load and Prepare Data


In [3]:
# Download sample data
ticker = 'AAPL'
stock_data = yf.download(ticker, start='2020-01-01', end='2023-12-31')
print(f"Downloaded {len(stock_data)} rows of data for {ticker}")
stock_data.head()
data_dict = {
    "Close": stock_data["Close"][ticker],
    "Open": stock_data["Open"][ticker],
    "High": stock_data["High"][ticker],
    "Low": stock_data["Low"][ticker],
    "Volume": stock_data["Volume"][ticker],
}
data = pd.DataFrame(data_dict)

[*********************100%***********************]  1 of 1 completed

Downloaded 1006 rows of data for AAPL


## 2. Basic Time Series Analysis
Let's analyze the components of our time series using decomposition.

In [4]:
# Initialize analyzer with configuration
config = TimeseriesConfig(
    decomposition_method='additive',
    seasonal_periods=[5, 21, 63, 252]  # Daily, Weekly, Monthly, Yearly
)
analyzer = TimeseriesAnalysis(data, config)

# Decompose the time series using daily seasonality (5 trading days)
decomposition = analyzer.decompose('Close', period=5)  # Using 5 for weekly trading pattern

# You can also analyze monthly seasonality
monthly_decomposition = analyzer.decompose('Close', period=21)  # Using 21 for monthly trading pattern

# Initialize visualizer
viz = TimeseriesVisualizer(analyzer)

# Plot decomposition
decomp_fig = viz.plot_decomposition('Close')
decomp_fig.show()

# Check for stationarity
stationarity = analyzer.analyze_stationarity('Close')
print("\nStationarity Analysis:")
print(f"ADF Test p-value: {stationarity['adf_test']['p_value']:.4f}")
print(f"Is stationary? {stationarity['adf_test']['is_stationary']}")


Stationarity Analysis:
ADF Test p-value: 0.5276
Is stationary? False


c:\apoorv\projects\specialized-viz\specialized_viz\timeseries\analysis.py:432: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


c:\Users\belgu\anaconda3\envs\github\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning:

'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.



## 3. Time Series Decomposition
Let's decompose our series into trend, seasonal, and residual components.

In [5]:
# Perform decomposition
decomp_fig = viz.plot_decomposition('Close')
decomp_fig.show()

# Plot trend analysis
trend_fig = viz.plot_trend_analysis('Close')
trend_fig.show()

## 4. Pattern Analysis
Now let's analyze various patterns in our data.

In [6]:
# Seasonal Analysis
seasonality = analyzer.analyze_seasonality('Close')
print("\nSeasonality Analysis:")
print(f"Monthly Seasonal Strength: {seasonality.get('seasonal_strength_21', 0):.4f}")

# Print holiday effects
print("\nHoliday Effects:")
for holiday, effect in seasonality['holiday_effects'].items():
    print(f"{holiday}: {effect:.4f}")

# Cycle Analysis
cycles = analyzer.analyze_cycles('Close')
print("\nDominant Cycles (days):")
periodic_lengths = cycles['periodicity']['periodic_lengths']
correlation_strength = cycles['periodicity']['correlation_strength']
for length, strength in zip(periodic_lengths, correlation_strength):
    print(f"Period: {length} days, Strength: {strength:.4f}")

# Print Fourier analysis results
print("\nFourier Analysis - Top frequencies:")
frequencies = cycles['fourier']['dominant_frequencies']
amplitudes = cycles['fourier']['amplitudes']
for freq, amp in zip(frequencies, amplitudes):
    if freq != 0:  # Avoid division by zero
        period = 1/freq
        print(f"Period: {period:.1f} days, Amplitude: {amp:.4f}")
        
# Plot correlogram
corr_fig = viz.plot_correlogram('Close')
corr_fig.show()

# Distribution Evolution
dist_fig = viz.plot_distribution_evolution('Close')
dist_fig.show()

c:\apoorv\projects\specialized-viz\specialized_viz\timeseries\analysis.py:119: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

c:\Users\belgu\anaconda3\envs\github\Lib\site-packages\pywt\_cwt.py:117: FutureWarning:

Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).




Seasonality Analysis:
Monthly Seasonal Strength: 0.0030

Holiday Effects:
is_weekend: 0.0000
month_end: 3.5674
fiscal_year_end: 13.5452

Dominant Cycles (days):
Period: 206 days, Strength: 0.5456
Period: 222 days, Strength: 0.5391
Period: 238 days, Strength: 0.5461

Fourier Analysis - Top frequencies:
Period: -503.0 days, Amplitude: 10884.4199
Period: 503.0 days, Amplitude: 10884.4199
Period: 1006.0 days, Amplitude: 14254.0813
Period: -1006.0 days, Amplitude: 14254.0813


c:\Users\belgu\anaconda3\envs\github\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning:

'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.



## 5. Change Point Detection and Anomaly Analysis
Let's identify significant changes and anomalies in our data.

In [7]:
# Detect change points
change_fig = viz.plot_change_points('Close')
change_fig.show()

# Detect anomalies
anomalies = analyzer.detect_anomalies('Close')
print("\nAnomalies detected by method:")
for method, anomaly_series in anomalies.items():
    print(f"{method}: {anomaly_series.sum()} anomalies")


KeyboardInterrupt: 

## 6. Forecasting
Let's implement various forecasting methods and compare their performance.

In [7]:
forecaster = TimeseriesForecasting(data)

# Create features
features = forecaster.create_features('Close')
print("\nFeatures created:", features.columns.tolist())

# Split data for testing
train_size = int(len(features) * 0.8)
train_features = features[:train_size]
test_features = features[train_size:]
train_target = data['Close'][:train_size]
test_target = data['Close'][train_size:]

# Try different forecasting methods
# 1. Seasonal Forecast
seasonal_result = forecaster.seasonal_forecast(train_features, train_target)

# 2. SARIMA Forecast
sarima_result = forecaster.sarima_forecast(train_features, train_target)

# 3. ETS Forecast
ets_result = forecaster.ets_forecast(train_features, train_target)

# 4. Combined Forecast
combined_result = forecaster.combined_seasonal_forecast(train_features, train_target)


Features created: ['lag_1', 'lag_7', 'lag_14', 'lag_30', 'rolling_mean_7', 'rolling_std_7', 'rolling_min_7', 'rolling_max_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_min_14', 'rolling_max_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_min_30', 'rolling_max_30', 'rolling_mean_90', 'rolling_std_90', 'rolling_min_90', 'rolling_max_90', 'month', 'day_of_week', 'day_of_year', 'week_of_year', 'is_month_end', 'is_month_start', 'is_quarter_end', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos', 'lag_diff_1_7', 'lag_diff_7_14', 'lag_diff_14_30']


NameError: name 'stats' is not defined

In [12]:
seasonal_result

{'forecast': 804    149.142549
 805    148.973856
 806    149.178945
 807    149.298144
 808    149.505257
 809    149.616635
 810    149.447941
 811    149.653031
 812    149.772230
 813    149.979343
 814    150.090720
 815    149.922027
 816    150.127117
 817    150.246316
 818    150.453429
 819    150.564806
 820    150.396113
 821    150.601203
 822    150.720401
 823    150.927515
 824    151.038892
 825    150.870199
 826    151.075289
 827    151.194487
 828    151.401600
 829    151.512978
 830    151.344285
 831    151.549375
 832    151.668573
 833    151.875686
 dtype: float64,
 'lower_bound': 804    143.835089
 805    143.666396
 806    143.871486
 807    143.990685
 808    144.197798
 809    144.309175
 810    144.140482
 811    144.345572
 812    144.464771
 813    144.671884
 814    144.783261
 815    144.614568
 816    144.819658
 817    144.938856
 818    145.145970
 819    145.257347
 820    145.088654
 821    145.293744
 822    145.412942
 823    145.620055
 824  

## 7. Model Evaluation and Comparison
Let's evaluate the performance of our different forecasting methods.

In [ ]:
# Compare all models
all_forecasts = {
    'Seasonal': seasonal_result['forecast'],
    'SARIMA': sarima_result['forecast'],
    'ETS': ets_result['forecast'],
    'Combined': combined_result['forecast']
}

# Evaluate each model
evaluation_results = {}
for model_name, forecast in all_forecasts.items():
    metrics = forecaster.evaluate_forecast(forecast, test_target)
    evaluation_results[model_name] = metrics

# Create comparison DataFrame
comparison_df = pd.DataFrame(evaluation_results).round(4)
print("\nModel Comparison:")
print(comparison_df)


## 8. Interactive Dashboard
Finally, let's create an interactive dashboard combining all our analyses.

In [ ]:
# Create comprehensive dashboard
dashboard = viz.create_comprehensive_dashboard('Close')
dashboard.show()

## 9. Advanced Analytics
Let's perform some advanced analytics on our time series.

In [ ]:
# Create animation of moving window analysis
animation_fig = viz.create_animation('Close')
animation_fig.show()

# Analyze feature importance
importance_fig = viz.plot_feature_importance(features, data['Close'])
importance_fig.show()

## 10. Online Learning and Adaptation
Finally, let's implement online learning to adapt to changing patterns.

In [ ]:
# Implement online learning
forecaster.online_learning(features, data['Close'], window_size=63)
print("\nOnline Learning Performance:")
print(forecaster.online_performance.describe().round(4))

## Conclusion
This comprehensive analysis demonstrates the wide range of capabilities in the specialized-viz time series module. Key findings include:
1. Seasonal patterns in the data
2. Significant change points
3. Forecasting model performance
4. Feature importance
5. Adaptation to changing patterns